In [6]:
import pandas as pd

import gc
####

import seaborn as sns
import plotly.express as px

import numpy as np

import sqlite3
from sqlite3 import Error
import os
import re, string
from nltk.corpus import stopwords
import nltk
import matplotlib.pyplot as plt

import torch
#from transformers import AutoTokenizer, BertForSequenceClassification
## felipe


from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA

nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


import pandas as pd
import re, string


pd.options.display.max_columns = None

#from sentence_transformers import SentenceTransformer, util
#from transformers import *

import spacy

nlp = spacy.load('es_core_news_lg', disable=['tagger', 'parser', 'ner'])



from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import preprocessing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
!python -m spacy download es_core_news_lg

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')


In [4]:
!python -m spacy download es_core_news_lg

%pip install pycorrector
%pip install unidecode
%pip install pyspellchecker

2023-07-17 15:58:50.485306: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
  Using cached https://github.com/explosion/spacy-models/releases/download/es_core_news_lg-3.5.0/es_core_news_lg-3.5.0-py3-none-any.whl (568.0 MB)
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')


# etl

In [ ]:
import pandas as pd
import sqlite3

In [7]:


query1 = "SELECT NUMERO_PQR,DESCRIPCION_PQR FROM pqr_priorizar where MEDIO_RECEPCION !=  'VIRTUAL' and unico=1 and SINTOMA_PQR ='CONCEPTOS FACTURADOS' and FECHA_CREACION >= '2023-07-10' and TECNOLOGIA ='FO'"

query2 = "(SELECT * FROM altas where ST_TIPO_SERVICIO='FTTH' AND FASE='ORDERF' AND (tipoTramite= 'VTA' OR tipoTramite=  'Sales')) AS myquery"

In [8]:

# Connect to the SQLite database file
conn = sqlite3.connect('/content/on2/pqr_priorizar.db')

# Read the SQL query into a pandas DataFrame

pqrs = pd.read_sql_query(query1, conn)

# Close the database connection
conn.close()

# Print the DataFrame
pqrs

#MDM-PQR-39191683

,NUMERO_PQR,DESCRIPCION_PQR
0,MDM-PQR-39816962,Clientese comunica por explicación de factura
1,MDM-PQR-39818282,Cliente LUIS FERNANDO RAGUA ESTRADA Cédula de ...
2,MDM-PQR-39819251,6016220682 SANDRA LILIANA SAAVEDRA ACERO 522...
3,MDM-PQR-39819280,NOMBRE : CLAUDIA CATALINA MEJIA LOPEZ CC: 5...
4,MDM-PQR-39819561,Linea: 6013150842 Cuenta: 7388032 Nombre: DEY...
...,...,...
1590,MDM-PQR-39935953,CEDULA: 1001287248 NOMBRE: KEVIN NICOLAS SAND...
1591,MDM-PQR-39936654,CC:1010213038 NOMBRE:FELIX ANTONIO GIRALDO SAL...
1592,MDM-PQR-39937069,"SOLICITUD, DEPLICADO DE FACTURACION"
1593,MDM-PQR-39937489,WHATSAPP// explicación de factura


# TRASNFORMAR LAS DATOS DE TEXTO

In [9]:
sys.path.append('../librerias')

In [10]:
from clean_text import clean_text

entro2


In [11]:
pqrs = pqrs.fillna("vacio")



In [12]:
pqrs.isnull().sum()

NUMERO_PQR         0
DESCRIPCION_PQR    0
dtype: int64

In [13]:
#declare answers and questions
descripcion=pqrs.DESCRIPCION_PQR





#clean_descripcion=[k for k in clean_descripcion if 'soporte'  in k]
clean_descripcion = [clean_text(des) for des in descripcion]


len(clean_descripcion)

1595

In [14]:
clean_descripcion[0]

'por eporplicación de factura'

spelling

In [15]:
from tqdm import tqdm
from spelling import correct_sentence


hola


In [16]:
corrected_sentences = []
for text in tqdm(clean_descripcion, desc="Processing"):
    corrected_sentence = correct_sentence(text)
    corrected_sentences.append(corrected_sentence)

clean_descripcion=corrected_sentences

Processing: 100%|██████████| 1595/1595 [04:33<00:00,  5.84it/s]


lemma token stemm

In [17]:
from lemastem import data_preprocessing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


['balvina', 'digna', 'jakeline', 'silver', 'ober', 'monrroy', 'nelson', 'melida', 'mazuera', 'tendríais', 'malo', 'traslaviña', 'lorena', 'lelio', 'fernand', 'ferrucho', 'januario', 'guacaneme', 'gamaliel', 'segismundo', 'baruc', 'creaciones', 'echavez', 'ossa', 'ipc', 'jhoam', 'lisney', 'ciudad', 'mayerling', 'alondra', 'espumas', 'guillermo', 'marivel', 'sequeda', 'derly', 'chaux', 'vidrios', 'zulay', 'deiny', 'claret', 'jhoana', 'cordoba', 'norvey', 'jacquelinne', 'ac', 'lilia', 'alcaldia', 'bersabe', 'pe&a', 'borraez', 'yurid', 'ecosistemas', 'larry', 'jeimy', 'saidy', 'heliberto', 'chivata', 'briseida', 'fausto', 'jhosep', 'yeiber', 'romero', 'margareth', 'aleira', 'giorgio', 'antony', 'wilbert', 'canro', 'jelber', 'vejarano', 'lennin', 'carbonell', 'adame', 'liney', 'rudolf', 'estos', 'farmacias', 'mendez', 'erisinda', 'turizo', 'sthefanny', 'viki', 'jesenia', 'casallas', 'ademir', 'jayder', 'quien', 'emilio', 'mega', 'sirlys', 'jeily', 'kristhian', 'silky', 'jannia', 'chirley', 

In [18]:

clean_descripcion=data_preprocessing(clean_descripcion)
clean_descripcion[190]

['solicitar', 'contar']

In [19]:
clean_descripcion[0]

['eporplicacion', 'factura']

In [20]:
merged_data=[]
for i in range(len(clean_descripcion)):
    merged_data.append(" ".join(clean_descripcion[i]))

merged_df=pd.DataFrame(merged_data,columns =['descripcion'])


In [21]:
from unidecode import unidecode

def remove_accents(text):
    return unidecode(text)

# Aplicar la función a la columna 'Texto'
merged_df['descripcion'] = merged_df['descripcion'].apply(remove_accents)

In [22]:
merged_df

,descripcion
0,eporplicacion factura
1,validar concepto facturar
2,validar factura
3,falla
4,solicitar reducir
...,...
1590,instalacion realizar facturacion llego cobrar ...
1591,facturacion
1592,solicitud delicado facturacion
1593,eporplicacion factura



clean_descripcion = [text for text in clean_descripcion if "svas" not in text and "directv" not in text  and "winsport" not in text and "hbo" not in text  ]

len(clean_descripcion)

In [23]:
from tfidf import TFIDF
import joblib

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from joblib import dump


X = merged_df['descripcion']
X_test=merged_df['descripcion']


X_train,_ = TFIDF(X,X_test)

# Create a new instance of PCA with the optimal number of components

pca = joblib.load('/content/PRIORIZAR/modelo/pca_binario.joblib')

#pca = PCA(n_components=585)


X_train_new = pca.fit_transform(X_train)


print("train with old features: ",np.array(X).shape)
print("train with new features:" ,np.array(X_train_new).shape)



tf-idf with 1102 features
train with old features:  (1595,)
train with new features: (1595, 804)


# cargar modelo

In [25]:
from joblib import dump

import joblib


model = joblib.load('/content/PRIORIZAR/modelo/modelo_binario.joblib')

print(model)

LGBMClassifier(bagging_fraction=0.9747337180903012, bagging_freq=4,
               feature_fraction=0.6526645750030313,
               lambda_l1=0.02873208934418634, lambda_l2=2.3446621696223424,
               learning_rate=0.0815126268370284, min_child_samples=35,
               n_estimators=88, num_leaves=35)


In [26]:
y_pred = model.predict(X_train_new)

pqrs['prediccion_algoritmo'] = y_pred

pqrs['descripcion_clean']=merged_df['descripcion']



# Lista de palabras a filtrar
words_to_remove = ['svas', 'directv', 'paramont', 'hbo']

# Filtrar las filas del DataFrame
c_facturados_correcion = pqrs[~pqrs['descripcion_clean'].str.contains('|'.join(words_to_remove))]

c_facturados_SVAS = pqrs[pqrs['descripcion_clean'].str.contains('|'.join(words_to_remove))]



c_facturados_correcion


,NUMERO_PQR,DESCRIPCION_PQR,prediccion_algoritmo,descripcion_clean
0,MDM-PQR-39816962,Clientese comunica por explicación de factura,1.0,eporplicacion factura
1,MDM-PQR-39818282,Cliente LUIS FERNANDO RAGUA ESTRADA Cédula de ...,1.0,validar concepto facturar
2,MDM-PQR-39819251,6016220682 SANDRA LILIANA SAAVEDRA ACERO 522...,1.0,validar factura
3,MDM-PQR-39819280,NOMBRE : CLAUDIA CATALINA MEJIA LOPEZ CC: 5...,1.0,falla
4,MDM-PQR-39819561,Linea: 6013150842 Cuenta: 7388032 Nombre: DEY...,1.0,solicitar reducir
...,...,...,...,...
1590,MDM-PQR-39935953,CEDULA: 1001287248 NOMBRE: KEVIN NICOLAS SAND...,1.0,instalacion realizar facturacion llego cobrar ...
1591,MDM-PQR-39936654,CC:1010213038 NOMBRE:FELIX ANTONIO GIRALDO SAL...,1.0,facturacion
1592,MDM-PQR-39937069,"SOLICITUD, DEPLICADO DE FACTURACION",1.0,solicitud delicado facturacion
1593,MDM-PQR-39937489,WHATSAPP// explicación de factura,1.0,eporplicacion factura


In [27]:
from datetime import date

today = date.today()

year, week_num, day_of_week = today.isocalendar()





In [28]:
errores = len(c_facturados_correcion[c_facturados_correcion['prediccion_algoritmo'] == 0])
total  =len(c_facturados_correcion)
sva=len(c_facturados_SVAS)
error=(errores/total)*100
semana=week_num-1
index='-'.join(map(str, [year,semana]))



data = {'errores': [errores],
        'total': [total],
        'sva': [sva],
        '%_error':[error],
        'semana':[semana],
        'index':[index]}

tracking_tipificacion = pd.DataFrame(data)


#tracking_c_facturado

In [30]:
file_path = "/content/PRIORIZAR/data/precision.txt"
with open(file_path, "r") as file:
    precision = file.read()


data["accuracy"]=precision


tracking_tipificacion

,errores,total,sva,%_error,semana,index
0,258,1550,45,16.645161,28,2023-28


cargando data apra tablero

In [31]:

 
# append data frame to CSV file
tracking_tipificacion.to_csv('/content/tracking_c_facturado.csv', mode='a', index=False, header=False)
 


!rclone copy  /content/tracking_c_facturado.csv one:/analitica_datos/tracking_c_facturado

In [32]:
import pandas as pd
from datetime import date


from datetime import datetime

# Obtener la fecha y hora actual
current_date = datetime.now()
# Get the current date
# Specify the root directory path
root_path = '/content/PRIORIZAR/SALIDAS/'

# Export the DataFrame to a CSV file with the root path and current date in the filename
filename1 = f"{root_path}c_facturados_correcion_{current_date}.csv"
filename2 = f"{root_path}c_facturados_SVAS_{current_date}.csv"


c_facturados_correcion.to_csv(filename1, index=False)

c_facturados_SVAS.to_csv(filename2, index=False)




#pandas_c_F.to_csv('c_facturados.csv', index=False)

print("DONE")

DONE


# modelo retipificacion

filter data with 0 label

In [33]:
retipificacion = c_facturados_correcion[c_facturados_correcion['prediccion_algoritmo'] == 0.0]
retipificacion

,NUMERO_PQR,DESCRIPCION_PQR,prediccion_algoritmo,descripcion_clean
9,MDM-PQR-39820457,Línea: Nombre de quien se comunica:Ingrid Pere...,0.0,validar envio factura detalle
15,MDM-PQR-39821454,Línea:6016332633 Nombre de quien se comunica:W...,0.0,validar genero pago final llegar factura
25,MDM-PQR-39822082,Cliente pide explicación de la ultima factura ...,0.0,pedir eporplicacion ultimo factura realizastir...
34,MDM-PQR-39823009,mal gestionado la pqr,0.0,mal gestionado
39,MDM-PQR-39823252,linea: 6012532166 nombre de quien se comunica:...,0.0,solicitar referente cobro pago cancelo indicado
...,...,...,...,...
1573,MDM-PQR-39933291,cliente:JESSICA DAMARIS AMAYA GONZALEZ cc:10...,0.0,clientejessica dama amar decir i pedir tarifa
1578,MDM-PQR-39934042,WILLIAM PINTO ROSAS //1119182808 //6014554736 ...,0.0,comunicar operadora indicar tenia facturacion ...
1582,MDM-PQR-39934377,NombreNEIR SANCHEZ Celular 3233308846 UENHog...,0.0,nombreneir uenhogar miny tipo segmento uenhoga...
1584,MDM-PQR-39934543,NOMBRE: MARIA FACNORY BUITRAGO GALLO - EDWARD...,0.0,informar dia ayer presentar ausencia telefonica


In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split



X = retipificacion['descripcion_clean']
X_test=retipificacion['descripcion_clean']


X_train,_ = TFIDF(X,X_test)

# Create a new instance of PCA with the optimal number of components

pca = joblib.load('/content/PRIORIZAR/modelo/pca_retipificacion.joblib')

#pca = PCA(n_components=361)


X_train_new = pca.fit_transform(X_train)


print("train with old features: ",np.array(X).shape)
print("train with new features:" ,np.array(X_train_new).shape)



tf-idf with 475 features
train with old features:  (258,)
train with new features: (258, 249)


In [35]:
model = joblib.load('/content/PRIORIZAR/modelo/retipificacion.joblib')

print(model)

LGBMClassifier(bagging_fraction=0.8902288478379407, bagging_freq=1,
               feature_fraction=0.9963731191377936,
               lambda_l1=0.31449684221997787, lambda_l2=0.028561307947605728,
               learning_rate=0.0988198318878283, min_child_samples=43,
               n_estimators=82, num_leaves=46)


In [36]:
y_pred = model.predict(X_train_new)

retipificacion["retipificacion_algoritmo"]=y_pred

<ipython-input-36-9505f33dc576>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retipificacion["retipificacion_algoritmo"]=y_pred


In [37]:
mapping = {0: 'BENEFICIO DE RETENCION', 1: 'INCREMENTOS DE FACTURA', 2: 'BAJA NO INGRESADA',  3: 'CANCELACION',4: 'COMPENSACION'}

# Separate features and target variable

retipificacion['tipificacion']= retipificacion['retipificacion_algoritmo'].replace(mapping)

retipificacion['index']=index


<ipython-input-37-660200b608e7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retipificacion['tipificacion']= retipificacion['retipificacion_algoritmo'].replace(mapping)
<ipython-input-37-660200b608e7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retipificacion['index']=index


In [38]:
file_path = "/content/PRIORIZAR/data/f1scoreretipificacion.txt"
df = pd.read_csv(file_path, sep=',', header=None)
f1 = df.values.flatten()

retipificacion.loc[retipificacion["retipificacion_algoritmo"] == 0, "f1-score"] = f1[0]
retipificacion.loc[retipificacion["retipificacion_algoritmo"] == 1, "f1-score"] = f1[1]
retipificacion.loc[retipificacion["retipificacion_algoritmo"] == 2, "f1-score"] = f1[2]
retipificacion.loc[retipificacion["retipificacion_algoritmo"] == 3, "f1-score"] = f1[3]
retipificacion.loc[retipificacion["retipificacion_algoritmo"] == 4, "f1-score"] = f1[4]


retipificacion

<ipython-input-38-273398139352>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retipificacion.loc[retipificacion["retipificacion_algoritmo"] == 0, "f1-score"] = f1[0]


,NUMERO_PQR,DESCRIPCION_PQR,prediccion_algoritmo,descripcion_clean,retipificacion_algoritmo,tipificacion,index,f1-score
9,MDM-PQR-39820457,Línea: Nombre de quien se comunica:Ingrid Pere...,0.0,validar envio factura detalle,0,BENEFICIO DE RETENCION,2023-28,0.92
15,MDM-PQR-39821454,Línea:6016332633 Nombre de quien se comunica:W...,0.0,validar genero pago final llegar factura,1,INCREMENTOS DE FACTURA,2023-28,0.93
25,MDM-PQR-39822082,Cliente pide explicación de la ultima factura ...,0.0,pedir eporplicacion ultimo factura realizastir...,1,INCREMENTOS DE FACTURA,2023-28,0.93
34,MDM-PQR-39823009,mal gestionado la pqr,0.0,mal gestionado,2,BAJA NO INGRESADA,2023-28,0.90
39,MDM-PQR-39823252,linea: 6012532166 nombre de quien se comunica:...,0.0,solicitar referente cobro pago cancelo indicado,3,CANCELACION,2023-28,0.89
...,...,...,...,...,...,...,...,...
1573,MDM-PQR-39933291,cliente:JESSICA DAMARIS AMAYA GONZALEZ cc:10...,0.0,clientejessica dama amar decir i pedir tarifa,2,BAJA NO INGRESADA,2023-28,0.90
1578,MDM-PQR-39934042,WILLIAM PINTO ROSAS //1119182808 //6014554736 ...,0.0,comunicar operadora indicar tenia facturacion ...,4,COMPENSACION,2023-28,0.93
1582,MDM-PQR-39934377,NombreNEIR SANCHEZ Celular 3233308846 UENHog...,0.0,nombreneir uenhogar miny tipo segmento uenhoga...,2,BAJA NO INGRESADA,2023-28,0.90
1584,MDM-PQR-39934543,NOMBRE: MARIA FACNORY BUITRAGO GALLO - EDWARD...,0.0,informar dia ayer presentar ausencia telefonica,2,BAJA NO INGRESADA,2023-28,0.90


In [39]:


retipificacion.to_csv('/content/retipificacion.csv', mode='a', index=False, header=False)
 
# print message

!rclone copy  /content/retipificacion.csv one:/analitica_datos/tracking_c_facturado

In [40]:
import pandas as pd
from datetime import date


from datetime import datetime

# Obtener la fecha y hora actual
current_date = datetime.now()
# Get the current date
# Specify the root directory path
root_path = '/content/PRIORIZAR/SALIDAS/'

# Export the DataFrame to a CSV file with the root path and current date in the filename
filename3 = f"{root_path}c_facturados_retipificacion_{current_date}.csv"



retipificacion.to_csv(filename3, index=False)





#pandas_c_F.to_csv('c_facturados.csv', index=False)

print("DONE")

DONE
